In [85]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import numpy as np
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
#Imports timer
import tqdm
#To do Regex
import re
#To do sentiment analysis
from textblob import TextBlob

import matplotlib.pyplot as plt 
import seaborn as sns

## Import data from CSV file to a DataFrame

In [86]:
twitter_df = pd.read_csv('Cleaned_Twitter_Scrape_10-12_2021.csv')
twitter_df.sample(5)

,Unnamed: 0,User,Date Created,Number of Likes,Tweet,Date,Hour,Sentiment,Numerical
38349,38349,crypto_uttam,2021-12-27 05:31:21+00:00,1,"DogemonGo, the only 100% Free AR Crypto Game i...",2021-12-27,5,positive,1
419205,419205,jeremyisaaclee,2021-12-05 02:34:16+00:00,0,$LUNA making new ATHs again a day after the fl...,2021-12-05,2,positive,1
472501,472501,bmurphypointman,2021-12-01 10:52:06+00:00,0,#business #bitcoin #tumblr #twitter #facebook ...,2021-12-01,10,neutral,0
925550,925550,NVAGUp,2021-11-01 02:13:37+00:00,0,⁦@doge_cola⁩ #dogebull #moontaste $SHIB $DOGE ...,2021-11-01,2,positive,1
254008,254008,dogesuppporteam,2021-12-23 11:13:46+00:00,73,#dogecoin #dogetothemoon #doge $doge 🔥🥰 https:...,2021-12-23,11,neutral,0


In [87]:
keep_columns = ['User', 'Date Created', 'Tweet', 'Date', 'Hour', 'Sentiment', 'Numerical']

twitter_df2 = twitter_df[keep_columns].copy()

twitter_df2


,User,Date Created,Tweet,Date,Hour,Sentiment,Numerical
0,DogecoinCuban,2021-12-31 23:59:58+00:00,@Rodolfo07351753 @sodogetip tip @Rodolfo073517...,2021-12-31,23,neutral,0
1,RNY1117,2021-12-31 23:59:47+00:00,HAPPY NEW DOGE YEAR,2021-12-31,23,positive,1
2,bojodus,2021-12-31 23:59:37+00:00,"Prayers Up for XRP, DOGE , SHIB \nx1000 to the...",2021-12-31,23,neutral,0
3,CryptoCapflow,2021-12-31 23:59:09+00:00,"[Spot] By 2021-12-31 14:00 GMT, DOGE total tra...",2021-12-31,23,neutral,0
4,CarlosS74671189,2021-12-31 23:59:04+00:00,@Darlene_Pizarro How much Doge would you like ...,2021-12-31,23,positive,1
...,...,...,...,...,...,...,...
1194430,FraxinusE,2021-10-20 15:16:21+00:00,@RichAvatar_ Doge,2021-10-20,15,neutral,0
1194431,realDogecoinOG,2021-10-20 15:16:19+00:00,Chart of Open Interests or the huge pump on pe...,2021-10-20,15,positive,1
1194432,jalantrading,2021-10-20 15:16:13+00:00,DOGE https://t.co/jmoMMslmTs,2021-10-20,15,neutral,0
1194433,LeviKrull,2021-10-20 15:16:05+00:00,Bought a doge miner with doge today,2021-10-20,15,neutral,0


In [88]:
#twitter_df_dates.set_index('Date', inplace=True)
#twitter_df_dates

In [54]:
twitter_df['lol'] = str(twitter_df['Date Created'].loc[0:3])

In [89]:
twitter_df2['Rounded_Date_Hour'] = [str[:-11]+'00:00' for str in twitter_df['Date Created']]

In [90]:
twitter_df2['Rounded_Date_Hour'] = pd.to_datetime(twitter_df2['Rounded_Date_Hour'])

In [47]:
twitter_df.groupby()

,Unnamed: 0,User,Date Created,Number of Likes,Tweet,Date,Hour,Sentiment,Numerical
0,0,DogecoinCuban,2021-12-31 23:59:58+00:00,1,@Rodolfo07351753 @sodogetip tip @Rodolfo073517...,2021-12-31,23,neutral,0
1,1,RNY1117,2021-12-31 23:59:47+00:00,2,HAPPY NEW DOGE YEAR,2021-12-31,23,positive,1
2,2,bojodus,2021-12-31 23:59:37+00:00,0,"Prayers Up for XRP, DOGE , SHIB \nx1000 to the...",2021-12-31,23,neutral,0
3,3,CryptoCapflow,2021-12-31 23:59:09+00:00,0,"[Spot] By 2021-12-31 14:00 GMT, DOGE total tra...",2021-12-31,23,neutral,0
4,4,CarlosS74671189,2021-12-31 23:59:04+00:00,1,@Darlene_Pizarro How much Doge would you like ...,2021-12-31,23,positive,1
...,...,...,...,...,...,...,...,...,...
1194430,1194430,FraxinusE,2021-10-20 15:16:21+00:00,0,@RichAvatar_ Doge,2021-10-20,15,neutral,0
1194431,1194431,realDogecoinOG,2021-10-20 15:16:19+00:00,4,Chart of Open Interests or the huge pump on pe...,2021-10-20,15,positive,1
1194432,1194432,jalantrading,2021-10-20 15:16:13+00:00,0,DOGE https://t.co/jmoMMslmTs,2021-10-20,15,neutral,0
1194433,1194433,LeviKrull,2021-10-20 15:16:05+00:00,0,Bought a doge miner with doge today,2021-10-20,15,neutral,0


In [103]:
sentiment_per_hour_df2 = twitter_df2.groupby(twitter_df2['Rounded_Date_Hour']).sum()

In [106]:
sentiment_per_hour_df2['Rounded_Date_Hour'] = sentiment_per_hour_df2.index

In [107]:
sentiment_per_hour_df2

,Hour,Numerical,Rounded_Date_Hour
Rounded_Date_Hour,,,
2021-10-20 15:00:00,7755,145,2021-10-20 15:00:00
2021-10-20 16:00:00,11616,191,2021-10-20 16:00:00
2021-10-20 17:00:00,11662,153,2021-10-20 17:00:00
2021-10-20 18:00:00,12420,153,2021-10-20 18:00:00
2021-10-20 19:00:00,14421,177,2021-10-20 19:00:00
...,...,...,...
2021-12-31 19:00:00,11742,140,2021-12-31 19:00:00
2021-12-31 20:00:00,13700,214,2021-12-31 20:00:00
2021-12-31 21:00:00,13062,180,2021-12-31 21:00:00


In [93]:
binance_df = pd.read_csv('bpfinf.csv')

In [77]:
#binance_df.set_index('Open time', inplace=True)

In [95]:
binance_df['Rounded_Date_Hour']=binance_df['Open time']

In [101]:
merged_df = pd.concat([sentiment_per_hour_df, binance_df], axis=1)

In [100]:
#inner_merge = pd.merge(sentiment_per_hour_df, binance_df, on='Rounded_Date_Hour', how='inner')
#inner_merge

In [119]:
lort= pd.concat([sentiment_per_hour_df, binance_df])

In [120]:
lort.tail(30)

,Unnamed: 0,Number of Likes,Hour,Numerical,Open time,Close,Rounded_Date_Hour
1675,1675,NaN,NaN,NaN,2021-12-28 19:00:00,0.1753,2021-12-28 19:00:00
1676,1676,NaN,NaN,NaN,2021-12-28 20:00:00,0.1762,2021-12-28 20:00:00
1677,1677,NaN,NaN,NaN,2021-12-28 21:00:00,0.1744,2021-12-28 21:00:00
1678,1678,NaN,NaN,NaN,2021-12-28 22:00:00,0.1741,2021-12-28 22:00:00
1679,1679,NaN,NaN,NaN,2021-12-28 23:00:00,0.1739,2021-12-28 23:00:00
1680,1680,NaN,NaN,NaN,2021-12-29 00:00:00,0.1754,2021-12-29 00:00:00
1681,1681,NaN,NaN,NaN,2021-12-29 01:00:00,0.1755,2021-12-29 01:00:00
1682,1682,NaN,NaN,NaN,2021-12-29 02:00:00,0.1765,2021-12-29 02:00:00
1683,1683,NaN,NaN,NaN,2021-12-29 03:00:00,0.1767,2021-12-29 03:00:00
1684,1684,NaN,NaN,NaN,2021-12-29 04:00:00,0.1756,2021-12-29 04:00:00


In [136]:
sentiment_per_hour_df.reset_index(inplace=True)

In [137]:
sentiment_per_hour_df

,Rounded_Date_Hour,Unnamed: 0,Number of Likes,Hour,Numerical
0,2021-10-20 15:00:00,617388992,1901,7755,145
1,2021-10-20 16:00:00,866520567,5392,11616,191
2,2021-10-20 17:00:00,818294071,8104,11662,153
3,2021-10-20 18:00:00,822590745,7228,12420,153
4,2021-10-20 19:00:00,904299924,4489,14421,177
...,...,...,...,...,...
1732,2021-12-31 19:00:00,58093854,3499,11742,140
1733,2021-12-31 20:00:00,64076357,3606,13700,214
1734,2021-12-31 21:00:00,58064011,5182,13062,180
1735,2021-12-31 22:00:00,61957584,8919,14696,132


In [157]:
sentiment_per_hour_df2 =sentiment_per_hour_df2[['Hour', 'Numerical']]


,Hour,Numerical
Rounded_Date_Hour,,
2021-10-20 15:00:00,7755,145
2021-10-20 16:00:00,11616,191
2021-10-20 17:00:00,11662,153
2021-10-20 18:00:00,12420,153
2021-10-20 19:00:00,14421,177
...,...,...
2021-12-31 19:00:00,11742,140
2021-12-31 20:00:00,13700,214
2021-12-31 21:00:00,13062,180


In [158]:
sentiment_per_hour_df2.reset_index(inplace=True)

In [159]:
sentiment_per_hour_df2

,Rounded_Date_Hour,Hour,Numerical
0,2021-10-20 15:00:00,7755,145
1,2021-10-20 16:00:00,11616,191
2,2021-10-20 17:00:00,11662,153
3,2021-10-20 18:00:00,12420,153
4,2021-10-20 19:00:00,14421,177
...,...,...,...
1732,2021-12-31 19:00:00,11742,140
1733,2021-12-31 20:00:00,13700,214
1734,2021-12-31 21:00:00,13062,180
1735,2021-12-31 22:00:00,14696,132


In [169]:
binance_df['Rounded_Date_Hour'] = pd.to_datetime(binance_df['Rounded_Date_Hour'])

In [130]:
type(binance_df['Rounded_Date_Hour'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [131]:
type(sentiment_per_hour_df['Rounded_Date_Hour'][0])

KeyError: 'Rounded_Date_Hour'

In [133]:
sentiment_per_hour_df2

,Hour,Numerical,Rounded_Date_Hour
Rounded_Date_Hour,,,
2021-10-20 15:00:00,7755,145,2021-10-20 15:00:00
2021-10-20 16:00:00,11616,191,2021-10-20 16:00:00
2021-10-20 17:00:00,11662,153,2021-10-20 17:00:00
2021-10-20 18:00:00,12420,153,2021-10-20 18:00:00
2021-10-20 19:00:00,14421,177,2021-10-20 19:00:00
...,...,...,...
2021-12-31 19:00:00,11742,140,2021-12-31 19:00:00
2021-12-31 20:00:00,13700,214,2021-12-31 20:00:00
2021-12-31 21:00:00,13062,180,2021-12-31 21:00:00


In [167]:
sentiment_per_hour_df2['Rounded_Date_Hour'] = pd.to_datetime(sentiment_per_hour_df2['Rounded_Date_Hour'])

In [170]:
sentiment_per_hour_df2.join(binance_df, on='Rounded_Date_Hour')

ValueError: You are trying to merge on datetime64[ns] and int64 columns. If you wish to proceed you should use pd.concat